# Load annotated data

In [75]:
import json
import pprint 
pp = pprint.PrettyPrinter(indent=4)
import numpy as np
import pandas as pd

# dataset_name = 'meme_retrieval_data'
# dataset_name = 'figmemes'
# dataset_name = 'figmemes_cot'
dataset_name = 'figmemes_cot_few_shot'
# dataset_name = 'memecap'

img_prompt_respond_file = ''
gt_data_file = ''
if dataset_name == 'meme_retrieval_data':
    img_prompt_respond_file = '../data/meme_retrieval_data/filtered_meme_configs_5_attributes_meme_retri.json'
elif dataset_name == 'figmemes':
    img_prompt_respond_file = '../data/figmemes/filtered_meme_configs_templatic_without_about_section.json'
    gt_data_file = '../data/figmemes/figmemes_annotations.tsv'
elif dataset_name == 'figmemes_cot':
    img_prompt_respond_file = '../data/figmemes/filtered_meme_configs_figmemes_cot.json'
    gt_data_file = '../data/figmemes/cot_samples_gt.tsv'
elif dataset_name == 'figmemes_cot_few_shot':
    img_prompt_respond_file = '../data/figmemes/filtered_meme_configs_1-11_cot_3_step_refined.json'
    gt_data_file = '../data/figmemes/cot_samples_gt.tsv'
elif  dataset_name == 'memecap':
    img_prompt_respond_file = '../data/memecap/filtered_meme_configs_5_attributes_memecap.json'

# load predicted data
meme_configs = []
with open(img_prompt_respond_file, 'r', encoding='utf-8') as json_file:
    meme_configs = json.load(json_file)
len(meme_configs)

# load ground truth data
gt_data = pd.read_csv(gt_data_file, sep='\t', index_col='img_name')

In [76]:
gt_data

,allusion,exaggeration,irony,anthrop,metaphor,contrast,year,arts,real,mixed,infograph
img_name,,,,,,,,,,,
1458964859691.jpg,1,0,0,0,0,0,2017,1,0,0,0
1464007930878.jpg,0,0,0,0,0,0,2020,0,1,0,0
1467472694126.png,0,0,1,0,0,0,2017,1,0,0,0
1468935750176.jpg,1,1,0,0,0,0,2019,1,0,0,0
1470534047335.jpg,0,0,0,0,0,0,2017,0,1,0,0
1470839669535.jpg,0,0,1,0,0,0,2017,1,0,0,0
1472045568054.jpg,0,0,0,0,0,0,2018,0,1,0,0
1472180356237.jpg,1,1,0,0,1,1,2017,0,0,1,0
1474915955788.png,0,0,0,0,0,1,2017,0,0,1,0


In [77]:
# gt_data.loc['01144951214.png'][:6].tolist()

In [78]:
def label2vec(label_list: list):
    labels = ['allusion', 'exaggeration', 'irony', 'anthrop', 'metaphor', 'contrast']
    vec = np.zeros((1, len(labels)))
    for label in label_list:
        vec[0, labels.index(label)] = 1
    return vec

label2vec(['exaggeration'])

array([[0., 1., 0., 0., 0., 0.]])

In [79]:
meme_configs[0]

{'image_dir': './data/figmemes/images/1458964859691.jpg',
 'prompt': '\n "You are a masterful, strict and critical assistant in interpretation of online memes, their style of literary devices, their meaning and humour.."\n \n <The list of choices and the definitions of literary devices>: \n **Allusion**: Referencing historical events, figures, symbols, art, literature or pop culture.\n **Exaggeration**: Similar to Hyperbole. Use of exaggerated terms for emphasis, including exaggerated visuals (including unrealistic features portraying minorities).\n **Irony**: Similar to Sarcasm. Use of words that convey a meaning that is the opposite of its usual meaning/mock someone or something with caustic or bitter use of words.\n **Anthropomorphism**: Similar to Zoomorphism. Attributing human qualities to animals, objects, natural phenomena or abstract concepts or applying animal characteristics to humans in a way that conveys additional meaning. \n **Metaphor**: Similar to Simile. Implicit or ex

In [80]:
predictions = []
gt = []
for idx, meme_conf in enumerate(meme_configs):
    print(idx, meme_conf)
    predictions.append(label2vec(meme_conf['literary device']))
    gt.append(gt_data.loc[meme_conf['image_dir'].split('/')[-1]][:6].tolist())

predictions = np.array(predictions).squeeze()
gt = np.array(gt)
predictions.shape, gt.shape

0 {'image_dir': './data/figmemes/images/1458964859691.jpg', 'prompt': '\n "You are a masterful, strict and critical assistant in interpretation of online memes, their style of literary devices, their meaning and humour.."\n \n <The list of choices and the definitions of literary devices>: \n **Allusion**: Referencing historical events, figures, symbols, art, literature or pop culture.\n **Exaggeration**: Similar to Hyperbole. Use of exaggerated terms for emphasis, including exaggerated visuals (including unrealistic features portraying minorities).\n **Irony**: Similar to Sarcasm. Use of words that convey a meaning that is the opposite of its usual meaning/mock someone or something with caustic or bitter use of words.\n **Anthropomorphism**: Similar to Zoomorphism. Attributing human qualities to animals, objects, natural phenomena or abstract concepts or applying animal characteristics to humans in a way that conveys additional meaning. \n **Metaphor**: Similar to Simile. Implicit or e

((11, 6), (11, 6))

In [81]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

print('macro:', f1_score(gt.flatten(), predictions.flatten(), average='macro'))
print('micro:', f1_score(gt.flatten(), predictions.flatten(), average='micro'))
print(classification_report(gt.flatten(), predictions.flatten()))

macro: 0.5568181818181819
micro: 0.6060606060606061
              precision    recall  f1-score   support

           0       0.86      0.60      0.70        52
           1       0.30      0.64      0.41        14

    accuracy                           0.61        66
   macro avg       0.58      0.62      0.56        66
weighted avg       0.74      0.61      0.64        66



In [82]:
def gimme_f1s(y_true, y_pred):
    print('zero')
    print(classification_report(y_true=y_true, y_pred=y_pred, zero_division=0))
    print('one')
    print(classification_report(y_true=y_true, y_pred=y_pred, zero_division=1))
    print()
    print('zero')
    f1s = ['micro', 'macro', 'weighted', 'samples']
    for score in f1s:
        f1 = f1_score(y_true=y_true, y_pred=y_pred, zero_division=0, average=score)*100
        print(score)
        print(f1)
        print()
    print('one')
    for score in f1s:
        f1 = f1_score(y_true=y_true, y_pred=y_pred, zero_division=1, average=score)*100
        print(score)
        print(f1)
        print()

gimme_f1s(gt, predictions)

zero
              precision    recall  f1-score   support

           0       0.38      0.75      0.50         4
           1       0.40      0.67      0.50         3
           2       0.25      1.00      0.40         2
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         2
           5       0.50      1.00      0.67         2

   micro avg       0.30      0.64      0.41        14
   macro avg       0.25      0.57      0.34        14
weighted avg       0.30      0.64      0.40        14
 samples avg       0.30      0.52      0.34        14

one
              precision    recall  f1-score   support

           0       0.38      0.75      0.50         4
           1       0.40      0.67      0.50         3
           2       0.25      1.00      0.40         2
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         2
           5       0.50      1.00      0.67         2

   micro avg  

In [83]:
print('\n "You are a masterful, strict and critical assistant in interpretation of online memes, their style of literary devices, their meaning and humour.."\n \n <The list of choices and the definitions of literary devices>: \n **Allusion**: Referencing historical events, figures, symbols, art, literature or pop culture.\n **Exaggeration**: Similar to Hyperbole. Use of exaggerated terms for emphasis, including exaggerated visuals (including unrealistic features portraying minorities).\n **Irony**: Similar to Sarcasm. Use of words that convey a meaning that is the opposite of its usual meaning/mock someone or something with caustic or bitter use of words.\n **Anthropomorphism**: Similar to Zoomorphism. Attributing human qualities to animals, objects, natural phenomena or abstract concepts or applying animal characteristics to humans in a way that conveys additional meaning. \n **Metaphor**: Similar to Simile. Implicit or explicit comparisons between two items or groups, attributing the properties of one thing to another. This category includes dehumanizing metaphors. \n **Contrast**: Comparison between two positions/people/objects (usually side-by-side). \n \n "<Multiple Choice> Please select one or multiple labels from the above list that are applied to the meme:"\n Your answer:\n \n "<Extraction of answer> Extract the suitable labels for the input meme and the multiple choice question above:"\n Your answer:\n \n "<Choice by choice comparison> Compare each of the exacted label with the meme and decide if this label could explain the meme:"\n Your answer:\n \n <Final labels>:\n {\n "literary device":["allusion", ...]\n }')


 "You are a masterful, strict and critical assistant in interpretation of online memes, their style of literary devices, their meaning and humour.."
 
 <The list of choices and the definitions of literary devices>: 
 **Allusion**: Referencing historical events, figures, symbols, art, literature or pop culture.
 **Exaggeration**: Similar to Hyperbole. Use of exaggerated terms for emphasis, including exaggerated visuals (including unrealistic features portraying minorities).
 **Irony**: Similar to Sarcasm. Use of words that convey a meaning that is the opposite of its usual meaning/mock someone or something with caustic or bitter use of words.
 **Anthropomorphism**: Similar to Zoomorphism. Attributing human qualities to animals, objects, natural phenomena or abstract concepts or applying animal characteristics to humans in a way that conveys additional meaning. 
 **Metaphor**: Similar to Simile. Implicit or explicit comparisons between two items or groups, attributing the properties of 